In [5]:
using Revise
using Test
using Random
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: TensorCI2, AtimesBinv, tensortrain, sitedims

import LinearAlgebra as LA

In [2]:
g = v -> exp(exp(1im * sum(v)))

#9 (generic function with 1 method)

In [10]:
function _reconst_tt(tt)
    sitedims_ = sitedims(tt)
    localdims = collect(prod.(sitedims_))
    r = [tt(collect(Tuple(i))) for i in CartesianIndices(Tuple(localdims))]
    returnsize = collect(Iterators.flatten(sitedims_))
    return reshape(r, returnsize...)
end
#
#_reconst_tt(tt)

_reconst_tt (generic function with 1 method)

In [33]:
localdims = (6, 6, 6, 6)
tolerance = 1e-8
allindices = CartesianIndices(localdims)

tci, ranks, errors = TCI.crossinterpolate1(ComplexF64, g, localdims; tolerance=tolerance)
tt = TCI.TensorTrain(tci)
ttr = TCI.reverse(tt)
@test TCI.rank(tci) == TCI.rank(tt)
@test TCI.rank(tci) == TCI.rank(ttr)
@test TCI.linkdims(tci) == TCI.linkdims(tt)
@test TCI.linkdims(tci) == reverse(TCI.linkdims(ttr))
gsum = ComplexF64(0.0)
for i in allindices
    @test TCI.evaluate(tci, i) ≈ TCI.evaluate(tt, i)
    @test TCI.evaluate(tci, i) ≈ TCI.evaluate(ttr, reverse(Tuple(i)))
    @test tt(i) == TCI.evaluate(tt, i)
    @test tt(i) ≈ TCI.evaluate(tt, reverse(Tuple(i)))
    functionvalue = g(Tuple(i))
    @test abs(TCI.evaluate(tt, i) - functionvalue) < tolerance
    @test abs(TCI.evaluate(ttr, reverse(Tuple(i))) - functionvalue) < tolerance
    gsum += functionvalue
end
@test gsum ≈ TCI.sum(tt)
@test gsum ≈ TCI.sum(ttr)

using LinearAlgebra
LA.norm(tt)

for method in [:SVD]
    ttcompressed = deepcopy(tt)
    TCI.compress!(ttcompressed, method; tolerance=0.0)
    #@show TCI.norm(ttcompressed - tt)
    @show TCI.norm(tt - tt)
    @show LA.norm(tt - tt)
    @show TCI.norm(tt)
    #@test TCI.rank(ttcompressed) <= 5
end

#for method in [:LU, :CI, :SVD]
    #ttcompressed = deepcopy(tt)
    #TCI.compress!(ttcompressed, method; tolerance=1.0)
    #@test TCI.rank(ttcompressed) <= TCI.rank(tt)
#end

TCI.norm(tt - tt) = 3.321893407454324e-27
LA.norm(tt - tt) = 3.321893407454324e-27
TCI.norm(tt) = 2952.595315384516


In [114]:
using Random
Random.seed!(1234)
T = Float64
N = 10
sitedims_ = [[2] for _ in 1:N]
χ = 10

tol = 0.1
bonddims = vcat(1, χ * ones(Int, N - 1), 1)
@show bonddims

tt = TCI.TensorTrain([
    randn(bonddims[n], sitedims_[n]..., bonddims[n + 1]) for n in 1:N
])

tt_compressed = deepcopy(tt)
TCI.compress!(tt_compressed, :SVD; tolerance=tol)
@show sqrt(LA.norm2(tt - tt_compressed)/LA.norm2(tt)) < sqrt(N) * tol

tt_compressed = deepcopy(tt)
TCI.compress!(tt_compressed, :SVD; tolerance=LA.norm(tt) * tol, normalizeerror=false)
@show sqrt(LA.norm2(tt - tt_compressed)/LA.norm2(tt)) < sqrt(N) * tol

bonddims = [1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1]
sqrt(LA.norm2(tt - tt_compressed) / LA.norm2(tt)) < sqrt(N) * tol = true
LA.norm(tt) * tol = 121572.37351022812
sqrt(LA.norm2(tt - tt_compressed) / LA.norm2(tt)) < sqrt(N) * tol = true


true

In [46]:
LA.norm2(tt)

2.4494897427831783

In [25]:
import TensorCrossInterpolation: rrLU, left, right

"""
Solve (LU) x = b

L: lower triangular matrix
U: upper triangular matrix
b: right-hand side vector

Return x
"""
function solve(L::Matrix{T}, U::Matrix{T}, b::Matrix{T}) where{T}
    N1, N2, N3 = size(L, 1), size(L, 2), size(U, 2)
    M = size(b, 2)

    all(triu(L, 1) .== 0) || error("L is not lower triangular")
    all(tril(U, -1) .== 0) || error("U is not upper triangular")
    
    # Solve Ly = b
    y = zeros(T, N2, M)
    for i in 1:N2
        y[i, :] .= b[i, :]
        for j = 1:i-1
            y[i, :] .-= L[i, j] .* y[j, :]
        end
        y[i, :] ./= L[i, i]
    end

    # Solve Ux = y
    x = zeros(T, N3, M)
    #for i in N3:-1:1
    for revi in 1:min(N3, N2)
        x[N3 - revi + 1, :] .= y[N2 - revi + 1, :]
        for j in N3-revi+2:N3
            x[N3 - revi + 1, :] .-= U[N2 - revi + 1, j] .* x[j, :]
        end
        x[N3 - revi + 1, :] ./= U[N2 - revi + 1, N3 - revi + 1]
    end

    return x
end

function Base.:\(A::rrLU{T}, b::AbstractMatrix{T}) where{T}
    b_perm = b[A.rowpermutation, :]
    x_perm = solve(A.L, A.U, b_perm)
    x = similar(x_perm)
    for i in 1:size(x, 1)
        x[A.colpermutation[i], :] .= x_perm[i, :]
    end
    return x
end

#@show lua.rowpermutation
#@show lua.colpermutation
@time lua \ b
#@show A \ b
#@test A * (lua \ b)  ≈ b

#@code_warntype solve(L, U, b)

  0.265198 seconds (327.06 k allocations: 18.486 MiB, 99.99% compilation time)


2×1 Matrix{Float64}:
 0.8267624667662581
 0.0

In [11]:
function Base.transpose(A::rrLU{T}) where{T}
    return rrLU(
        A.colpermutation, A.rowpermutation,
        Matrix(transpose(A.U)), Matrix(transpose(A.L)), !A.leftorthogonal, A.npivot, A.error)
end

tlua = transpose(lua)

TCI.left(tlua) * TCI.right(tlua) ≈ transpose(A)

true

In [14]:
Random.seed!(1234)
N1, N2, N3 = 100, 100, 100
A = rand(N1, N2)

tlu = transpose(TCI.rrlu(A))

@test TCI.left(tlu) * TCI.right(tlu) ≈ A'

Test Passed

In [80]:
A = L * U
lua = TCI.rrlu(A)
@show TCI.left(lua) * TCI.right(lua) ≈ A
@show lua.rowpermutation
@show lua.colpermutation

TCI.left(lua) * TCI.right(lua) ≈ A = true
lua.rowpermutation = [2, 1]
lua.colpermutation = [2, 1]


2-element Vector{Int64}:
 2
 1

In [81]:
@show A

A = [0.016551426583781177 0.08767964216628267; 0.05493752065675756 0.359912199470381]


2×2 Matrix{Float64}:
 0.0165514  0.0876796
 0.0549375  0.359912

In [86]:
import TensorCrossInterpolation: rrLU, left, right

function solve(A::rrLU{T}, b::AbstractMatrix{T}) where{T}
    b_perm = b[A.rowpermutation, :]
    x_perm = solve(A.L, A.U, b_perm)
    return x_perm[A.colpermutation, :]
end

solve(lua, b) ≈ A \ b


true